<h1>Retrieving Financial Data from EIKON Refinitiv through API</h1>

<p> This notebook offers a useful starting point for those interested in retrieving financial data from <strong>EIKON Refinitiv</strong> (formerly <strong>Reuters</strong>) through <strong>API</strong>.</p>
<p> The procedure relies on the Python package <strong>eikon</strong>. Make sure to have it installed before starting, together with pandas. An alternative is to use the <strong>refinitiv-data</strong> library. </p>
<p> To provide an example of usage, the notebook retrieves data for ESG variables (emission data). You can adjust the API requests to retrieve whatever type of financial data available in EIKON, according to your needs. </p>
<p> What is crucial to know at the very beginning is that the amount of data that can be retrieved per request through the EIKON API is very limited. Additionally, the EIKON API is quite unstable. This forces us to download the data in several steps and through a robust loop. </p>


<h2>1. Setting up libraries, API connection and working paths</h2>

<h3>Importing libraries</h3>

In [ ]:
import pandas as pd
import eikon as ek 
#import refinitiv.data as rd
print('Eikon package version: ', ek.__version__)
#print('Refinitiv-data package version: ', rd.__version__)

<h3>Setting up API connection to EIKON</h3>

Here you should set up the app key which can be generated inside the EIKON app using the AppKey Generator.

In [ ]:
ek.set_app_key('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')

<h3>Defining working paths</h3>

In [ ]:
# Local path used to store and retrieve company identifiers for data retrieval
company_list_path = r'D:\Data\Climate risks\Company list\companies_full_list.csv'
# Output path for data retrieved in Section 4
output_path = r'D:\Data\Climate risks\Firm-level data\Emissions\2023_capex_tangible_cash_esg_score.csv'

<h2>2. [Test] Example provided by LSEG guide on EIKON API access</h2>

<p>This is an example taken from the official LSEG starter guide for API access to EIKON data to illustrate how the API works and to test it.</p>
For more info, please see: https://developers.lseg.com/en/api-catalog/eikon/eikon-data-api/quick-start.

In [ ]:
test, err = ek.get_data(['GOOG.O', 'MSFT.O', 'FB.O', 'AMZN.O', 'TWTR.K'], 
                        ['TR.Revenue.date','TR.Revenue','TR.GrossProfit'],
                        {'Scale': 6, 'SDate': 0, 'EDate': -2, 'FRQ': 'FY', 'Curn': 'EUR'})
test

<h2>3. Retrieving a list of companies for which desired data are available</h2>

<p>First of all, we need to retrieve a list for which Refinitiv has data on emissions in order to run our API query. Such a list can be retrieved using the app SCREENER in Refinitiv. 
As of January 2024, we have over 7800 (7839) companies in the Refinitiv database with data for CO2 emissions. Given the limit of 5000 companies for the screener app, we will import two different list of companies - one for companies with HQ in European or American countries (4570) and one for the rest (3269). </p>

<p> In case you plan to retrieve a large set of variables for the same corporates, <strong> store your company identifiers to skip this step later </strong>. In our setting, we retrieve them only once and we start again in Section 4 from the saved identifiers.</p>

For more info on the use of the SCREENER app, see: https://developers.lseg.com/en/article-catalog/article/find-your-right-companies-with-screener-eikon-data-apis-python

<h4>First retrieval of company identifiers using SCREENER</h4>

In [ ]:
#Output from SCREENER app
#For companies with HQ in Europe or in the Americas
#=@TR("SCREEN(U(IN(Equity(active,public,private,primary))/*UNV:PublicPrivate*/), TR.CO2EmissionTotal(Period=FY0)>=0.01, IN(TR.HQCountryCode,""AL"",""AD"",""BA"",""HR"",""CY"",""GI"",""GR"",""IT"",""MK"",""MT"",""PT"",""ME"",""RS"",""SM"",""S"&"I"",""ES"",""VA"",""AT"",""BE"",""FR"",""DE"",""LI"",""LU"",""MC"",""NL"",""CH"",""BY"",""BG"",""CZ"",""HU"",""MD"",""PL"",""RO"",""RU"",""SK"",""UA"",""DK"",""EE"",""FO"",""FI"",""GG"",""IS"",""IE"",""IM"",""JE"",""LV"",""LT"",""N"&"O"",""SJ"",""SE"",""GB"",""AI"",""AG"",""AR"",""AW"",""BS"",""BB"",""BZ"",""BO"",""BR"",""KY"",""CL"",""CO"",""CR"",""CU"",""CW"",""DM"",""DO"",""EC"",""SV"",""FK"",""GF"",""GD"",""GP"",""GT"",""GY"",""HT"",""HN"",""JM"",""MQ"",""M"&"X"",""MS"",""NI"",""PA"",""PY"",""PE"",""PR"",""BL"",""KN"",""LC"",""MF"",""VC"",""SX"",""SR"",""TT"",""TC"",""UY"",""VE"",""VG"",""VI"",""BM"",""CA"",""GL"",""PM"",""US""), CURN=EUR)","TR.CommonName;TR.CO2EmissionTotal(Period=FY0)"&";TR.HeadquartersCountry","curn=EUR RH=In CH=Fd")
#For companies with HQ in Africa, Oceania and Asia
#=@TR("SCREEN(U(IN(Equity(active,public,private,primary))/*UNV:PublicPrivate*/), TR.CO2EmissionTotal(Period=FY0)>=0.01, IN(TR.HQCountryCode,""AF"",""BD"",""BT"",""IO"",""IN"",""IR"",""MV"",""NP"",""PK"",""LK"",""AM"",""AZ"",""BH"",""GE"",""I"&"Q"",""IL"",""JO"",""KW"",""LB"",""OM"",""PS"",""QA"",""SA"",""SY"",""TR"",""AE"",""YE"",""BN"",""KH"",""ID"",""LA"",""MY"",""MM"",""PH"",""SG"",""TH"",""TL"",""VN"",""CN"",""HK"",""JP"",""KP"",""KR"",""MO"",""MN"",""TW"",""KZ"",""K"&"G"",""TJ"",""TM"",""UZ"",""DZ"",""EG"",""LY"",""MA"",""SS"",""SD"",""TN"",""EH"",""AO"",""CM"",""CF"",""TD"",""CG"",""CD"",""GQ"",""GA"",""ST"",""BJ"",""BF"",""CV"",""GM"",""GH"",""GN"",""GW"",""CI"",""LR"",""ML"",""MR"",""NE"",""N"&"G"",""SH"",""SN"",""SL"",""TG"",""BW"",""LS"",""NA"",""ZA"",""SZ"",""BI"",""KM"",""DJ"",""ER"",""ET"",""KE"",""MG"",""MW"",""MU"",""YT"",""MZ"",""RE"",""RW"",""SC"",""SO"",""TZ"",""UG"",""ZM"",""ZW"",""AU"",""CX"",""CC"",""NZ"",""N"&"F"",""CK"",""PF"",""NU"",""PN"",""WS"",""AS"",""TK"",""TO"",""TV"",""WF"",""FJ"",""NC"",""PG"",""SB"",""VU"",""GU"",""KI"",""MH"",""FM"",""NR"",""MP"",""PW"",""UM""), CURN=EUR)","TR.CommonName;TR.CO2EmissionTotal(Period=FY0);TR.Hea"&"dquartersCountry","curn=EUR RH=In CH=Fd")

#test on a smaller sample
#italian_companies = "SCREEN(U(IN(Equity(active,public,private,primary))), TR.CO2EmissionTotal(Period=FY0)>=0.01, IN(TR.HQCountryCode,""IT""), CURN=USD)"

#IMPORTANT TO MAKE THIS WORK starting from the EIKON Excel extension output
#From the Excel formula, remove /*UNV:PublicPrivate*/ (comment) and a character "&" in the middle of the list of countries

#Formula for list of companies with headquarter in Europe or Americas (4570)
eu_am_hq_formula = "SCREEN(U(IN(Equity(active,public,private,primary))), TR.CO2EmissionTotal(Period=FY0)>=0.01, IN(TR.HQCountryCode,""AL"",""AD"",""BA"",""HR"",""CY"",""GI"",""GR"",""IT"",""MK"",""MT"",""PT"",""ME"",""RS"",""SM"",""SI"",""ES"",""VA"",""AT"",""BE"",""FR"",""DE"",""LI"",""LU"",""MC"",""NL"",""CH"",""BY"",""BG"",""CZ"",""HU"",""MD"",""PL"",""RO"",""RU"",""SK"",""UA"",""DK"",""EE"",""FO"",""FI"",""GG"",""IS"",""IE"",""IM"",""JE"",""LV"",""LT"",""NO"",""SJ"",""SE"",""GB"",""AI"",""AG"",""AR"",""AW"",""BS"",""BB"",""BZ"",""BO"",""BR"",""KY"",""CL"",""CO"",""CR"",""CU"",""CW"",""DM"",""DO"",""EC"",""SV"",""FK"",""GF"",""GD"",""GP"",""GT"",""GY"",""HT"",""HN"",""JM"",""MQ"",""MX"",""MS"",""NI"",""PA"",""PY"",""PE"",""PR"",""BL"",""KN"",""LC"",""MF"",""VC"",""SX"",""SR"",""TT"",""TC"",""UY"",""VE"",""VG"",""VI"",""BM"",""CA"",""GL"",""PM"",""US""), CURN=EUR)"

#Formula for list of companies with headquarter in Africa, Asia or Oceania (3269)
row_formula = "SCREEN(U(IN(Equity(active,public,private,primary))), TR.CO2EmissionTotal(Period=FY0)>=0.01, IN(TR.HQCountryCode,""AF"",""BD"",""BT"",""IO"",""IN"",""IR"",""MV"",""NP"",""PK"",""LK"",""AM"",""AZ"",""BH"",""GE"",""IQ"",""IL"",""JO"",""KW"",""LB"",""OM"",""PS"",""QA"",""SA"",""SY"",""TR"",""AE"",""YE"",""BN"",""KH"",""ID"",""LA"",""MY"",""MM"",""PH"",""SG"",""TH"",""TL"",""VN"",""CN"",""HK"",""JP"",""KP"",""KR"",""MO"",""MN"",""TW"",""KZ"",""KG"",""TJ"",""TM"",""UZ"",""DZ"",""EG"",""LY"",""MA"",""SS"",""SD"",""TN"",""EH"",""AO"",""CM"",""CF"",""TD"",""CG"",""CD"",""GQ"",""GA"",""ST"",""BJ"",""BF"",""CV"",""GM"",""GH"",""GN"",""GW"",""CI"",""LR"",""ML"",""MR"",""NE"",""NG"",""SH"",""SN"",""SL"",""TG"",""BW"",""LS"",""NA"",""ZA"",""SZ"",""BI"",""KM"",""DJ"",""ER"",""ET"",""KE"",""MG"",""MW"",""MU"",""YT"",""MZ"",""RE"",""RW"",""SC"",""SO"",""TZ"",""UG"",""ZM"",""ZW"",""AU"",""CX"",""CC"",""NZ"",""NF"",""CK"",""PF"",""NU"",""PN"",""WS"",""AS"",""TK"",""TO"",""TV"",""WF"",""FJ"",""NC"",""PG"",""SB"",""VU"",""GU"",""KI"",""MH"",""FM"",""NR"",""MP"",""PW"",""UM""), CURN=EUR)"

<h4>Using the SCREENER formula to retrieve company identifiers - Part 1</h4>

In [ ]:
#Retrieving a list of names for companies based in Europe or Americas

eu_am_companies_data, e = ek.get_data(eu_am_hq_list, ek.TR_Field('TR.CompanyName'))
eu_am_companies = eu_am_companies_data['Instrument'].tolist()

#Retrieving associated countrycodes via API and then merging with previous dataframe with company names
eu_am_companies_cc, e = ek.get_data(eu_am_companies, ek.TR_Field('TR.HQCountryCode'))

eu_am_companies_data = pd.merge(eu_am_companies_data, eu_am_companies_cc, 'left', 'Instrument')

#export obtained dataset as .csv file
eu_am_companies_data.to_csv(r'D:\Data\Climate risks\Company list\eu_am_company_list.csv', index = False)

display(eu_am_companies_data)

<h4>Using the SCREENER formula to retrieve company identifiers - Part 2</h4>

In [ ]:
#Retrieving a list of names for companies based in Africa, Asia or Oceania

row_companies_data, e = ek.get_data(row_formula, ek.TR_Field('TR.CompanyName'))
row_companies = row_companies_data['Instrument'].tolist()

#Retrieving associated countrycodes via API and then merging with previous dataframe with company names
row_companies_cc, e = ek.get_data(row_companies, ek.TR_Field('TR.HQCountryCode'))

row_companies_data = pd.merge(row_companies_data, row_companies_cc, 'left', 'Instrument')

#export obtained dataset as .csv file
row_companies_data.to_csv(r'D:\Data\Climate risks\Company list\row_company_list.csv', index = False)

display(row_companies_data)

<h4>Extracting full list of companies and saving it for later use</h4>

In [ ]:
list_companies = pd.concat([eu_am_companies_data, row_companies_data])
list_companies = list_companies.sort_values('Country ISO Code of Headquarters', ascending=True)

#export obtained dataset as .csv file
list_companies.to_csv(company_list_path, index = False)

display(list_companies)

In [ ]:
del(test)
del(eu_am_hq_formula, eu_am_companies, eu_am_companies_cc, eu_am_companies_data)
del(row_formula, row_companies, row_companies_cc, row_companies_data)

<h2>4. Querying Data through EIKON API</h2>

<h3>a. Importing list of companies for which we want to retrieve data</h3>

In [ ]:
# This list was retrieved using Section 3 of the notebook
companies_data = pd.read_csv(company_list_path)
company_codes = companies_data['Instrument'].tolist()
print(company_codes[0:5])
print(len(company_codes))

<h3>b. Define fields to be downloaded and test connection to EIKON</h3>

In [ ]:
#Companies for which we want to retrieve data were previously retrieved and imported 
#Define variables to retrieve - here a set of possible corporate variables of interest


fields = [#ek.TR_Field('TR.CO2EstimationMethod', {'SDate':'FY2017'}),               #CO2 Estimation Method
          #ek.TR_Field('TR.TargetsEmissions', {'SDate':'FY2023'}),
          #ek.TR_Field('TR.F.PeriodEndDate', {'Period':'FY2022'}),#Does it have an emission reduction target?
          #ek.TR_Field('TR.CO2DirectScope1', {'Period':'FY2022'}),                   #CO2 Equivalent Scope1 Emissions
          #ek.TR_Field('TR.CO2IndirectScope2', {'Period':'FY2022'}),                 #CO2 Equivalent Scope2 Emissions
          #ek.TR_Field('TR.CO2EmissionTotal', {'Period':'FY2022'})                 #CO2 Equivalent Emissions Total
          #ek.TR_Field('TR.Revenue', {'Scale': 6, 'Period':'FY2022', 'Curn': 'EUR'}),                     #Revenue
          #ek.TR_Field('TR.Revenue.date', {'SDate':'FY2018'})
          #ek.TR_Field('TR.AnalyticCO2', {'SDate':'FY2018'}),                      #Total CO2 E.E. to Revenues in USD million
          #ek.TR_Field('TR.CO2EmissionTotal.periodenddate', {'SDate':'FY2018'}),   #EOP for CO2 Equivalent Emissions Total
          #ek.TR_Field('TR.NAICSIndustryGroupCode'),
          #ek.TR_Field('TR.NACEClassification'),
          #ek.TR_Field('TR.LegalEntityIdentifier'),
          #ek.TR_Field('TR.CreditRatioPD'),
          #ek.TR_Field('TR.CreditComboCPD'),
          #ek.TR_Field('TR.TotalAssetsReported', {'Scale': 6, 'Period':'FY2022', 'Curn': 'EUR'}),
          #ek.TR_Field('TR.TotalLiabilities', {'Scale': 6, 'Period':'FY2022', 'Curn': 'EUR'}),
          #ek.TR_Field('TR.F.EBITDAMargPct', {'Period':'FY2022'})
          ek.TR_Field('TR.CapitalExpenditures', {'Scale': 6, 'Period':'FY2022', 'Curn': 'EUR'}),
          ek.TR_Field('TR.PropertyPlantEquipmentTotalNet', {'Scale': 6, 'Period':'FY2022', 'Curn': 'EUR'}),
          ek.TR_Field('TR.CashAndSTInvestments', {'Scale': 6, 'Period':'FY2022', 'Curn': 'EUR'}),
          ek.TR_Field('TR.TRESGScore', {'Period':'FY2022'}),
          #ek.TR_Field('TR.TRESGScore', {'SDate':'FY2021'}),
          #ek.TR_Field('TR.TRESGScore', {'SDate':'FY2022'}),
          #ek.TR_Field('TR.OrganizationStatusCode'),
          #ek.TR_Field('TR.InstrumentListingStatus'),
          #ek.TR_Field('TR.TargetsEmissions', {'Period':'FY2022'}),
          #ek.TR_Field('TR.EmissionReductionTargetYear', {'Period':'FY2022'}),
          #ek.TR_Field('TR.EmissionReductionTargetPctage', {'Period':'FY2022'}),
          #ek.TR_Field('TR.CSRReportingScope', {'Period':'FY2022'})
          #ek.TR_Field('TR.F.PeriodEndDate', {'EDate':'FY2018'})
    
    ]

# Test connection
try:
    print("Testing connection...")
    df, e = ek.get_data('ALDAR.AD', fields)
    display(df)
    if df.isnull().values.any()==False:         #check if we actually downloaded all data requested
        print("Eikon is working!")
    else:
        print("EIKON IS NOT WORKING!")
except EikonError:
    print("EIKON IS NOT WORKING!")

In [ ]:
# Use first company to retrieve column names and create empty dataframe that will be appended by the loop
df_template=df.drop([0]).copy()
df_template

<h3>c. Define function and retrieve data - one row per request</h3>

In [ ]:
# Consider using Section D to retrieve data much more efficiently
# Define a function to retrieve data efficiently from EIKON, given a list of company codes, fields to be downloaded and output path

def force_get_eikon_data(company_codes, fields, output_path):
    
    #use first company to retrieve column names and create empty dataframe - hard-coded
    #df, e = ek.get_data('ALDAR.AD', fields)
    #df_template=df.drop([0])
    

    #creating a while loop to overcome the issues with the API (Timeout and BadRequest errors)
    total_completed_cases = list()
    to_be_downloaded = company_codes.copy()
    df = df_template.copy()
    df_to_save = df_template.copy()
    df.to_csv(output_path, index=False, mode='w', header=True) #create the empty csv file where we will append data

    while (len(to_be_downloaded)!=0):                    #run the loop as long as there are companies for which data is not downloaded
    
        completed_cases = list()
        print("Launching loop...")
     
        for company in to_be_downloaded:
            try:
                temp, e = ek.get_data(company, fields)   # download data

                df = pd.concat([df, temp])                       #append data to dataframe
                df_to_save = pd.concat([df_to_save, temp])       #append data to temporary dataframe needed to save data 
                total_completed_cases.append(company)            #append to list of completed cases
                completed_cases.append(company)                  #append to temporary list of completed cases
                to_be_downloaded.remove(company)                 #remove from list of companies to be downloaded
                
                print("Downloaded data for " + str(company)) 
            except:
                if len(completed_cases)>0:
                    df_to_save.to_csv(output_path, index=False, mode='a', header=False) #save what we downloaded so far             
                    print("EikonError after " + str(len(completed_cases)) + " companies. Saving data...")

                else:
                    print("EikonError after " + str(len(completed_cases)) + " companies.")
                                      
                print("EikonError after " + str(len(completed_cases)) + " companies. Saving data...")
                print("Remaining companies: " + str(len(to_be_downloaded)))
                print("Relaunching...")
                completed_cases = list()
                df_to_save = df_template.copy()
                break

    df_to_save.to_csv(output_path, index=False, mode='a', header=False) #save data
    print("Downloaded and saved " + str(len(total_completed_cases)) + " rows of data")
    print(df.tail())
    return df
    
force_get_eikon_data(company_codes, fields, output_path)

<h3>d. Define function and retrieve data - three rows per request</h3>

In [ ]:
#define a function to retrieve data efficiently from EIKON, given a list of company codes, fields to be downloaded and output path
#Three rows per request

def grouped(iterable, n):
    "s -> (s0,s1,s2,...sn-1), (sn,sn+1,sn+2,...s2n-1), (s2n,s2n+1,s2n+2,...s3n-1), ..."
    return zip(*[iter(iterable)]*n)

def open_or_create_csv(filepath):
    #Check whether a partial csv file exists, otherwise create the empty csv file where we will append data
    #and define list of firms for which we have to download data
    to_be_downloaded = []
    try:
        df = pd.read_csv(filepath)
        downloaded_firms = df.iloc[:,0].unique().tolist()
        for company in company_codes:
            if company not in downloaded_firms:
                to_be_downloaded.append(company)
        print("Found existing file with data for", str(len(downloaded_firms)), "firms!")
        print("To be downloaded:", str(len(to_be_downloaded)))
        
    except FileNotFoundError:
        #Save first copy if file is not there and download all companies
        print("Printing new file...")
        df = df_template.copy()
        df.to_csv(filepath, index=False, mode='w', header=True)
        to_be_downloaded = company_codes.copy()
        
    except Exception as e:
        print("An error occured:", e)
        
    return df, to_be_downloaded

def check_correct_data_structure(df):
    try:
        df.columns.tolist() == df_template.columns.tolist()
    except:
        raise Exception("Columns of the loaded dataframe do not match fields to download! Check your filepath or EIKON fields!")
        
def force_get_eikon_data_3x(company_codes, fields, output_path):
    
    #use first company to retrieve column names and create empty dataframe - hard-coded
    #df, e = ek.get_data('ALDAR.AD', fields)
    #df_template=df.drop([0])
    
    #creating a while loop to overcome the issues with the API (Timeout and BadRequest errors)
    total_completed_cases = list()
    df_to_save = df_template.copy()
        
    #Check whether a partial csv file exists, otherwise create the empty csv file where we will append data
    df, to_be_downloaded = open_or_create_csv(output_path)
    check_correct_data_structure(df)

    while (len(to_be_downloaded)!=0):                    #run the loop as long as there are companies for which data is not downloaded
    
        completed_cases = list()
        print("Launching loop...")
        
        while (len(to_be_downloaded) >= 3):                          #group by 3 as long as we have at least 3 elements           
            for company1, company2, company3 in grouped(to_be_downloaded, 3):
                try:
                    temp, e = ek.get_data([company1, company2, company3], fields)   # download data
                    print(temp)

                    df = pd.concat([df, temp])                       #append data to dataframe
                    df_to_save = pd.concat([df_to_save, temp])       #append data to temporary dataframe needed to save data

                    for company in [company1, company2, company3]:
                        total_completed_cases.append(company)            #append to list of completed cases
                        completed_cases.append(company)                  #append to temporary list of completed cases
                        to_be_downloaded.remove(company)                 #remove from list of companies to be downloaded
                
                    print("Downloaded data for " + str(company1) + ", " + str(company2) + ", " + str(company3)) 
                except:
                    df_to_save.to_csv(output_path, index=False, mode='a', header=False) #print what we downloaded so far
                                      
                    print("EikonError after " + str(len(completed_cases)) + " companies. Saving data...")
                    print("Remaining companies: " + str(len(to_be_downloaded)))
                    print("Relaunching...")
                    completed_cases = list()
                    df_to_save = df_template.copy()
                    break

        for company in to_be_downloaded:
            try:
                temp, e = ek.get_data(company, fields)   # download data

                df = pd.concat([df, temp])                       #append data to dataframe
                df_to_save = pd.concat([df_to_save, temp])       #append data to temporary dataframe needed to save data 
                total_completed_cases.append(company)            #append to list of completed cases
                completed_cases.append(company)                  #append to temporary list of completed cases
                to_be_downloaded.remove(company)                 #remove from list of companies to be downloaded
                
                print("Downloaded data for " + str(company)) 
            except:
                if len(completed_cases)>0:
                    df_to_save.to_csv(output_path, index=False, mode='a', header=False) #save what we downloaded so far             
                    print("EikonError after " + str(len(completed_cases)) + " companies. Saving data...")

                else:
                    print("EikonError/User stop after " + str(len(completed_cases)) + " companies.")

                print("Remaining companies: " + str(len(to_be_downloaded)))
                print("Relaunching...")
                completed_cases = list()
                df_to_save = df_template.copy()
                break

    df_to_save.to_csv(output_path, index=False, mode='a', header=False) #save data
    print("Downloaded and saved " + str(len(total_completed_cases)) + " rows of data")
    print(df.tail())
    return df
    
force_get_eikon_data_3x(company_codes, fields, output_path)